In [16]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from arch import arch_model

step 1: implement volatility model (modified GARCH)  
  
$\sigma_{n}^{2} = \gamma V_{lt} +\alpha u_{n-1}^{2} + \beta\sigma_{n-1}^{2}$  
  
where:  
  
$\sigma_{n}$ is the volatility of the market variable for day $n$ (made at the end of day $n-1$). $\sigma_{n}^{2}$ is the variance of the market variable for day $n$  
$\gamma$ is the weight assigned to $V_{lt}$  
$V_{lt}$ is the long-run average variance rate  
$\alpha$ is the weight assigned to $u_{n-1}^{2}$   
$u_{n-1}$ is the percentage change in the market variable between the end of day $n-2$ and the end of day $n-1$ (the most recent daily percentage change in the market variable). $u_{n-1}^{2}$ is the squared return of the market variable between the end of day $n-2$ and the end of day $n-1$  

$u_{n-1} = \frac{S_{n-1} - S_{n-2}}{S_{n-2}}$  

$S_{n-1}$ is the value of the market variable at the end of day $n-1$  
$S_{n-2}$ is the value of the market variable at the end of day $n-2$  

$\beta$ is the weight assigned to $\sigma_{n-1}^{2}$  
$\sigma_{n-1}$ is the volatility of the market variable for day $n-1$ (made at the end of day $n-2$) . $\sigma_{n-1}^{2}$ is the variance of the market variable for day $n-1$  
  
since the weights must sum to unity, it follows that  
  
$\gamma+\alpha+\beta=1$  
  
setting $\omega=\gamma V_{lt}$, the model can also be written as  
  
$\sigma_{n}^{2} = \omega +\alpha u_{n-1}^{2} + \beta\sigma_{n-1}^{2}$  
  
we include one lag of an asymmetric shock which transforms a GARCH model into a GJR-GARCH model with variance dynamics given by  
  
$\sigma^2_n   =  \omega + \alpha u_{n-1}^2 + \gamma u_{n-1}^2 I_{[u_{n-1}<0]}+ \beta \sigma_{n-1}^2$

where $I$ an indicator function that takes the value 1 when its argument is true. the log likelihood improves substantially with the introduction of an asymmetric term, and the parameter estimate is highly significant.  

to improve the fit a little more, we model the volatility using absolute values rather than a variance process that evolves in squares. this process, known as a TARCH/ZARCH model is given by  
  
$\sigma_n  =  \omega + \alpha \left|u_{n-1}\right| + \gamma \left|u_{n-1}\right| I_{[u_{n-1}<0]}+ \beta \sigma_{n-1}$

one final adjustment, financial returns are often heavy tailed, and a Student’s T distribution is a simple method to capture this feature. The call to arch changes the distribution from a Normal to a Students’s T.

In [ ]:
def volatility(df):
    # df: dataframe of price data. uses GJR-GARCH/TARCH (power=1) + t distribution.
    df = df.copy()
    returns = 100 * df["Close"].pct_change().dropna()
    df = df.iloc[1:].copy()

    am = arch_model(returns, p=1, o=1, q=1, power=1.0, dist="studentst")
    result = am.fit()
    df.loc[:, "Volatility"] = result._volatility
    df.loc[:, "Rolling Volatility"] = df["Volatility"].rolling(window=10).mean()

    calculated = df["Volatility"].mean()
    expected = np.var(returns)
    print(f"percent error in volatility calculation: {(np.abs(calculated - expected) / expected) * 100}")
    return df

step 2: momentum  
don't have much to say about this it's pretty straightforward

In [ ]:
def absolute_momentum(df):
    # 4 month momentum
    df = df.copy()
    df['Momentum'] = (df["Close"] / df["Close"].shift(84)) - 1
    df.dropna()
    return df

step 3: ATR Trend/Breakout system  
produces a signal based on bands. if a given's day high is higher than the upper band, the following day the model will go long (signal = 2). if a given day's low is lower than the lower band, the following day the model will go neutral/short (signal = -2). the bands are defined by the 42-day rolling average true range, with the true range being given by  
  
$TR = max\left[(H-L), \left|H-C_{n-1}\right|, \left|L-C_{n-1}\right|\right]$  
  
where $H$ is the day's high, $L$ is the day's low, and $C_{n-1}$ is the previous close

In [ ]:
def atr_trend_breakout(df):
    df = df.copy()
    high = df["High"]
    low = df["Low"]
    close = df["Close"]
    tr = pd.concat([high - low, (high - close.shift(1)).abs(), (low - close.shift(1)).abs()], axis=1).max(axis=1)
    atr = tr.rolling(42).mean()
    upper = high.rolling(42).max() + atr  # upper = session highs + ATR
    lower = low.rolling(42).min() - atr
    signal = pd.Series(0.0, index=df.index)
    long_mask = high > upper
    short_mask = low < lower
    signal.loc[long_mask] = 2
    signal.loc[short_mask] = -2
    df['ATR Trend/Breakout'] = signal.replace(0, np.nan).ffill().fillna(0)
    return df

step 4: average relative correlations  
4 months average correlation across the ETFs on daily returns. requires proper merged dataframe but thats for me to figure out later

In [ ]:
def correlation(returns_df):
    tickers = returns_df.columns.tolist()
    out = pd.DataFrame(index=returns_df.index, columns=tickers, dtype=float)
    for i in range(84, len(returns_df) + 1):
        chunk = returns_df.iloc[i - 84:i]
        corr = chunk.corr()
        idx = returns_df.index[i - 1]
        for col in tickers:
            others = [c for c in tickers if c != col]
            if others:
                out.loc[idx, col] = corr.loc[col, others].mean()
    return out

In [ ]:
def ranks_for_raam(momentum_series, vol_series, corr_series):
    """
    Rank 1..N with higher = better. M: higher momentum = higher rank. V,C: lower vol/corr = higher rank.
    Returns (rank_m, rank_v, rank_c) as Series.
    """
    from scipy.stats import rankdata
    # Rank(M): 11 = best (highest momentum) -> rankdata(-M) gives 1 to best, so we want 1..11 with 11=best -> 12 - rankdata(-M) no: rankdata(-M) gives 1 to max M. So rankdata(-M) is 1 for best. Paper: "ranked 1 to 11 in ascending order of M" so lowest M gets 1. So rankdata(M) gives 1 to lowest. So Rank(M)=rankdata(M) means lowest M gets 1. But "greater momentum = greater rank" so we need Rank(M) = rankdata(-momentum): 1 for lowest M, 11 for highest M. So 11 = best. Good.
    rank_m = pd.Series(rankdata(-momentum_series), index=momentum_series.index)
    # Rank(V): lower vol = higher rank. rankdata(vol) gives 1 to lowest vol. So 1=best. We want 11=best so Rank(V) = 12 - rankdata(vol).
    n = len(vol_series)
    rank_v = pd.Series(n + 1 - rankdata(vol_series), index=vol_series.index)
    rank_c = pd.Series(n + 1 - rankdata(corr_series), index=corr_series.index)
    return rank_m, rank_v, rank_c

def ranked_allocation(rank_m, rank_v, rank_c, w_m=1/3, w_v=1/3, w_c=1/3, top_n=5, cash_ticker="SHY"):
    """
    Total Rank = wM*(N+1-Rank(M)) + wV*(N+1-Rank(V)) + wC*(N+1-Rank(C)) so lower total = better.
    Select top_n assets with lowest Total Rank. Exclude cash_ticker from selection.
    """
    tickers = [x for x in rank_m.index if x != cash_ticker]
    n = len(tickers)
    total = w_m * (n + 1 - rank_m.reindex(tickers).fillna(0))
    total += w_v * (n + 1 - rank_v.reindex(tickers).fillna(0))
    total += w_c * (n + 1 - rank_c.reindex(tickers).fillna(0))
    top = total.nsmallest(top_n).index.tolist()
    return top

def raam_weights(momentum_series, rank_m, rank_v, rank_c, w_m=1/3, w_v=1/3, w_c=1/3, top_n=5, cash_ticker="SHY"):
    """
    RAAM allocation: top 5 by total rank; if momentum < 0 replace with cash.
    momentum_series: last-day momentum for each ticker (e.g. at month-end).
    Returns dict ticker -> weight (sum 1.0).
    """
    top = ranked_allocation(rank_m, rank_v, rank_c, w_m, w_v, w_c, top_n, cash_ticker)
    weight_per = 1.0 / top_n
    weights = {}
    cash_weight = 0.0
    for t in top:
        if t in momentum_series and momentum_series[t] < 0:
            cash_weight += weight_per
        else:
            weights[t] = weight_per
    if cash_weight > 0:
        weights[cash_ticker] = weights.get(cash_ticker, 0) + cash_weight
    return weights

tickers in our portfolio:
**Brazilian Equities:**  
EWZ  
FLBR  
EWZS  
  
**International Equities:**  
EFA  
EEM  
  
**Brazilian Real Estate:**  
HGBS11  
BTLG11  
  
****Brazilian Natural Resources – Commodities:**
BIAU39
BSLV39
BCOM39

Brazilian Bonds:
BLTN
VWOB (Emerging Markets)

International Bonds:
IGOV

Cash:
SHY


In [ ]:
TICKERS = ["EWZ", "FLBR", "EWZS", "EFA", "EEM", "HGBS11", "BTLG11", "BIAU39", "BSLV39", "BCOM39", "BLTN", "VWOB", "IGOV", "SHY"]
RISKY_TICKERS = [t for t in SEVEN_TWELVE_TICKERS if t != "SHY"]
CASH_TICKER = "SHY"

In [ ]:
def load_7twelve_data(start="2005-01-01", end="2025-01-01"):
    """Download OHLC for 7Twelve tickers and align to common index."""
    data = {}
    for t in SEVEN_TWELVE_TICKERS:
        try:
            d = yf.download(t, start=start, end=end, progress=False, auto_adjust=True)
            if isinstance(d.columns, pd.MultiIndex):
                d.columns = d.columns.droplevel("Ticker")
            if "Close" in d.columns and len(d) > 100:
                data[t] = d[["Open", "High", "Low", "Close"]].copy()
        except Exception as e:
            print(t, e)
    # Align to common dates
    common = None
    for t, df in data.items():
        idx = df.index
        common = idx if common is None else common.intersection(idx)
    for t in list(data.keys()):
        data[t] = data[t].loc[common].ffill().bfill()
    return data

In [ ]:
def build_raam_panel(data_dict, verbose=True):
    """
    Build panel of M, V (10d smoothed vol), C for each ticker.
    data_dict: ticker -> OHLC DataFrame. Uses pipeline's volatility model (GARCH) per asset.
    """
    returns_dict = {}
    vol_dict = {}
    mom_dict = {}
    for t, df in data_dict.items():
        if df is None or len(df) < 252:
            continue
        if verbose:
            print(f"Fitting volatility for {t} ...")
        try:
            vdf = volatility(df)
            returns_dict[t] = vdf["Close"].pct_change()
            vol_dict[t] = vdf["Rolling Volatility"]
            mom_dict[t] = absolute_momentum(vdf)
        except Exception as e:
            if verbose:
                print(t, e)
    # Align
    common = None
    for t in returns_dict:
        idx = returns_dict[t].dropna().index
        common = idx if common is None else common.intersection(idx)
    returns_df = pd.DataFrame({t: returns_dict[t].reindex(common).ffill().bfill() for t in returns_dict})
    vol_df = pd.DataFrame({t: vol_dict[t].reindex(common).ffill().bfill() for t in vol_dict})
    mom_df = pd.DataFrame({t: mom_dict[t].reindex(common).ffill().bfill() for t in mom_dict})
    if verbose:
        print("Computing average relative correlation ...")
    corr_df = average_relative_correlation(returns_df)
    return returns_df, vol_df, mom_df, corr_df

In [ ]:
def raam_backtest(returns_df, vol_df, mom_df, corr_df, w_m=1/3, w_v=1/3, w_c=1/3, top_n=5, cash_ticker=None):
    """
    Monthly backtest: at each month-end compute RAAM weights, hold next month.
    Returns (monthly_returns_series, cumulative_series).
    """
    from scipy.stats import rankdata
    cash_ticker = cash_ticker or CASH_TICKER
    risky = [c for c in returns_df.columns if c != cash_ticker]
    month_ends = returns_df.resample("ME").last().index
    monthly_ret = returns_df.resample("ME").apply(lambda x: (1 + x).prod() - 1)
    strategy_ret = []
    dates = []
    for i, me in enumerate(month_ends):
        if me not in vol_df.index or me not in mom_df.index:
            continue
        v = vol_df.loc[me].reindex(risky).dropna()
        m = mom_df.loc[me].reindex(risky).dropna()
        c = corr_df.loc[me].reindex(risky).dropna()
        common = v.index.intersection(m.index).intersection(c.index).tolist()
        if len(common) < 5:
            continue
        n_risky = len(common)
        rank_m = pd.Series(rankdata(-m[common]), index=common)
        rank_v = pd.Series(n_risky + 1 - rankdata(v[common]), index=common)
        rank_c = pd.Series(n_risky + 1 - rankdata(c[common]), index=common)
        top = ranked_allocation(rank_m, rank_v, rank_c, w_m, w_v, w_c, top_n, cash_ticker)
        weight_per = 1.0 / top_n
        weights = {}
        cash_w = 0.0
        for t in top:
            if m.get(t, 0) < 0:
                cash_w += weight_per
            else:
                weights[t] = weight_per
        if cash_w > 0:
            weights[cash_ticker] = weights.get(cash_ticker, 0) + cash_w
        next_me = month_ends[month_ends > me].min() if (month_ends > me).any() else None
        if next_me is None:
            break
        ret_next = 0.0
        for t, w in weights.items():
            if t in monthly_ret.columns and next_me in monthly_ret.index:
                r = monthly_ret.loc[next_me, t]
                ret_next += w * (r if pd.notna(r) else 0)
        strategy_ret.append(ret_next)
        dates.append(next_me)
    strat_series = pd.Series(strategy_ret, index=pd.DatetimeIndex(dates)).dropna()
    cum = (1 + strat_series).cumprod()
    return strat_series, cum

In [ ]:
# Run RAAM strategy (uses pipeline volatility model)
# Use shorter period and fewer tickers for a quicker run; expand as needed.
START, END = "2010-01-01", "2024-12-01"
tickers_subset = ["SPY", "EFA", "VNQ", "DBC", "AGG", "BWX", "VGK", "VWO", "TIP", "LQD", "SHY"]  # 11 assets

data = load_7twelve_data(start=START, end=END)
data = {t: data[t] for t in tickers_subset if t in data and data[t] is not None}
returns_df, vol_df, mom_df, corr_df = build_raam_panel(data, verbose=True)
# Backtest
monthly_ret, cum = raam_backtest(returns_df, vol_df, mom_df, corr_df)
print("RAAM monthly returns (last 12):", monthly_ret.tail(12))
print("Cumulative return:", cum.iloc[-1] - 1 if len(cum) else None)

In [ ]:
# Plot RAAM vs buy-and-hold SPY (monthly)
if len(cum) > 0 and "SPY" in returns_df.columns:
    spy_monthly = (1 + returns_df["SPY"]).resample("ME").prod() - 1
    spy_cum = (1 + spy_monthly).cumprod()
    ax = cum.reindex(spy_cum.index).ffill().plot(label="RAAM", figsize=(10, 4))
    spy_cum.reindex(cum.index).ffill().plot(ax=ax, label="SPY")
    plt.legend()
    plt.title("RAAM (2018 Dow Award strategy) vs SPY")
    plt.ylabel("Cumulative return")
    plt.show()

TODO:
implement momentum
implement ATR trend / breakout system
implement average relative correlations